In [1]:
from IPython.display import HTML
HTML('''
    <style> body {font-family: "Roboto Condensed Light", "Roboto Condensed";} h2 {padding: 10px 12px; background-color: #E64626; position: static; color: #ffffff; font-size: 40px;} .text_cell_render p { font-size: 15px; } .text_cell_render h1 { font-size: 30px; } h1 {padding: 10px 12px; background-color: #E64626; color: #ffffff; font-size: 40px;} .text_cell_render h3 { padding: 10px 12px; background-color: #0148A4; position: static; color: #ffffff; font-size: 20px;} h4:before{ 
    content: "@"; font-family:"Wingdings"; font-style:regular; margin-right: 4px;} .text_cell_render h4 {padding: 8px; font-family: "Roboto Condensed Light"; position: static; font-style: italic; background-color: #FFB800; color: #ffffff; font-size: 18px; text-align: center; border-radius: 5px;}input[type=submit] {background-color: #E64626; border: solid; border-color: #734036; color: white; padding: 8px 16px; text-decoration: none; margin: 4px 2px; cursor: pointer; border-radius: 20px;}</style>
''')

# 👥 Group Information

<p><strong>Code Language:</strong> <span style="font-size:18px;">Python</span></p>

<table style="font-size:18px;">
  <tr>
    <th>Name</th>
    <th>SID</th>
  </tr>
  <tr>
    <td>Ngoc Minh Dao</td>
    <td>520577590</td>
  </tr>
  <tr>
    <td>Manh Duc Nguyen</td>
    <td>520561337</td>
  </tr>
</table>

# Sydney Public Transport Analysis

### Import necessary libraries

In [2]:
# Imports for spatial data
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import time
import numpy as np
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement

# Imports for pgadmin
from sqlalchemy import create_engine, text
import psycopg2
import psycopg2.extras
import json
from sqlalchemy import text

### Connect to pgAdmin

In [3]:
credentials = "Credentials.json"

def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [4]:
db, conn = pgconnect(credentials)

Connected successfully.


### SRID Setup

In [5]:
srid = 4326

In [6]:
def create_wkt_element(geom, srid):
    if geom.geom_type == 'Polygon':
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

## Task 1: Import and Clean datasets

### 1.1 Load + clean Sydney Trains data

#### 1.1.1 Load data

In [7]:
# Load the file
trains = gpd.read_file("SydneyTrainRoutes/sydneytrains/SydneyTrains.shp")
trains = trains.rename(columns={"st_length(": "st_length"})
print(trains.columns)
# trains.head(5)

Index(['objectid', 'shape_id', 'route_id', 'agency_id', 'route_shor',
       'route_long', 'route_desc', 'route_type', 'route_colo', 'route_text',
       'exact_time', 'route_ty00', 'st_length', 'geometry'],
      dtype='object')


In [8]:
# Find all the Sydney trains
sydney_trains = trains[trains["agency_id"].str.startswith("Sydney")]
# sydney_trains.head(5)

In [9]:
# Check data types
sydney_trains.dtypes

objectid         int64
shape_id        object
route_id        object
agency_id       object
route_shor      object
route_long      object
route_desc      object
route_type      object
route_colo      object
route_text      object
exact_time      object
route_ty00      object
st_length      float64
geometry      geometry
dtype: object

#### 1.1.2 SRID Transformation

In [10]:
sydney_trains = sydney_trains.copy()  # creating a copy of the original for later
sydney_trains['geom'] = sydney_trains['geometry'].apply(lambda x: create_wkt_element(geom=x,srid=srid))  # applying the function
sydney_trains = sydney_trains.drop(columns="geometry")  # deleting the old copy
# sydney_trains.head(5)

In [11]:
# Check for missing values 
null_val = sydney_trains.isnull().sum()
print(f"Missing values per column:\n{null_val}")

Missing values per column:
objectid       0
shape_id       0
route_id       0
agency_id      0
route_shor     0
route_long     0
route_desc     0
route_type     0
route_colo     0
route_text     0
exact_time    54
route_ty00     0
st_length      0
geom           0
dtype: int64


In [12]:
# For "exact_time", fill with "Unknown"
sydney_trains['exact_time'] = sydney_trains['exact_time'].fillna('Unknown')
# sydney_trains.head(5)

#### 1.1.3 Create table for query

In [13]:
conn.execute(text("""
DROP TABLE IF EXISTS sydney_trains;
CREATE TABLE sydney_trains (
    objectid     INTEGER PRIMARY KEY,
    shape_id     VARCHAR(255),
    route_id     VARCHAR(255),
    agency_id    VARCHAR(255),
    route_shor   VARCHAR(255),
    route_long   VARCHAR(255),
    route_desc   VARCHAR(255),
    route_type   VARCHAR(255),
    route_colo   VARCHAR(255),
    route_text   VARCHAR(255),
    exact_time   VARCHAR(255),
    route_ty00   VARCHAR(255),
    st_length    DOUBLE PRECISION,
    geom         GEOMETRY(LINESTRING, 4326)
);
"""))

In [14]:
sydney_trains.to_sql(
    'sydney_trains', conn, if_exists='append', index=False, dtype={'geom': Geometry('LINESTRING', srid)}
)

54

In [15]:
# -- Create a spatial index on the 'geom' column for efficient spatial queries
conn.execute(text("""
CREATE INDEX idx_geom ON sydney_trains USING GIST (geom);
"""))

In [16]:
query(conn, "SELECT * FROM sydney_trains LIMIT 5;")

,objectid,shape_id,route_id,agency_id,route_shor,route_long,route_desc,route_type,route_colo,route_text,exact_time,route_ty00,st_length,geom
0,1,APS_1a,APS_1a,SydneyTrains,T8,City Circle to Macarthur via Airport,T8 Airport & South Line,Rail,00954C,FFFFFF,Unknown,Rail,67399.109358,0102000020E6100000000100007C832F76740D704138EF...
1,62,T3_1a,T3_1a,SydneyTrains,T3,City Circle to Liverpool via Regents Park,T3 Liverpool & Inner West Line,Rail,F37021,FFFFFF,Unknown,Rail,45833.184944,0102000020E6100000E70000002497FFAB740D704188F4...
2,2,APS_1c,APS_1c,SydneyTrains,T8,City Circle to Macarthur via Sydenham,T8 Airport & South Line,Rail,00954C,FFFFFF,Unknown,Rail,64669.017674,0102000020E6100000EC0000007C832F76740D704138EF...
3,3,APS_1e,APS_1e,SydneyTrains,T8,City Circle to Leppington via Sydenham,T8 Airport & South Line,Rail,00954C,FFFFFF,Unknown,Rail,57993.335454,0102000020E6100000F50000007C832F76740D704138EF...
4,4,APS_1f,APS_1f,SydneyTrains,T8,City Circle to Leppington via Sydenham,T8 Airport & South Line,Rail,00954C,FFFFFF,Unknown,Rail,57991.813538,0102000020E6100000F50000002497FFAB740D704188F4...


### 1.2 Load + clean Train Station Entrance Locations data

#### 1.2.1 Load data

In [17]:
# Load the file
entrance_loc = pd.read_csv("TrainStationEntranceLocations/stationentrances2020_v4.csv")
entrance_loc.columns = [col.lower() for col in entrance_loc.columns]
print(entrance_loc.columns)

Index(['train_station', 'street_name', 'street_type', 'entrance_type', 'lat',
       'long', 'exit_number'],
      dtype='object')


In [18]:
# Check data types
entrance_loc.dtypes

train_station     object
street_name       object
street_type       object
entrance_type     object
lat              float64
long             float64
exit_number      float64
dtype: object

In [19]:
# Check for missing values 
null_val = entrance_loc.isnull().sum()
print(f"Missing values per column:\n{null_val}")

Missing values per column:
train_station       0
street_name         0
street_type        28
entrance_type       0
lat                 0
long                0
exit_number      1024
dtype: int64


In [20]:
# For Street_Type, fill with "Unknown"
entrance_loc['street_type'] = entrance_loc['street_type'].fillna('Unknown')
# For Exit_Number, assuming missing = 0 exits
entrance_loc['exit_number'] = entrance_loc['exit_number'].fillna(0).astype(int)
entrance_loc.head(5)

,train_station,street_name,street_type,entrance_type,lat,long,exit_number
0,Aberdeen,Macqueen,St,Ramp,-32.166886,150.891957,0
1,Aberdeen,Macqueen,St,Stairs,-32.166900,150.891975,0
2,Adamstown,Park,Ave,Path,-32.933706,151.720452,0
3,Adamstown,Park,Ave,Path,-32.933827,151.720236,0
4,Adamstown,St James,Rd,Stairs,-32.933414,151.720363,0


In [21]:
entrance_loc['geom'] = gpd.points_from_xy(entrance_loc.long, entrance_loc.lat)      # creating the geometry column
entrance_loc = entrance_loc.drop(columns = ['long', 'lat'])        # removing the old latitude/longitude fields
entrance_loc.head(5)

,train_station,street_name,street_type,entrance_type,exit_number,geom
0,Aberdeen,Macqueen,St,Ramp,0,POINT (150.89196 -32.16689)
1,Aberdeen,Macqueen,St,Stairs,0,POINT (150.89198 -32.1669)
2,Adamstown,Park,Ave,Path,0,POINT (151.72045 -32.93371)
3,Adamstown,Park,Ave,Path,0,POINT (151.72024 -32.93383)
4,Adamstown,St James,Rd,Stairs,0,POINT (151.72036 -32.93341)


#### 1.2.2 SRID Transformation

In [22]:
entrance_loc['geom'] = entrance_loc['geom'].apply(lambda x: WKTElement(x.wkt, srid=srid))
entrance_loc.dtypes

train_station    object
street_name      object
street_type      object
entrance_type    object
exit_number       int32
geom             object
dtype: object

#### 1.2.3 Create table for query

In [23]:
conn.execute(text("""
DROP TABLE IF EXISTS train_entrance;
CREATE TABLE train_entrance (
    train_station  VARCHAR(255),
    street_name    VARCHAR(255),
    street_type    VARCHAR(255),
    entrance_type  VARCHAR(255),
    exit_number    INTEGER,
    geom           GEOMETRY(POINT, 4326)
);
"""))

In [24]:
entrance_loc.to_sql('train_entrance', conn, if_exists='append', index=False, dtype={'geom': Geometry('POINT', srid)})

74

In [25]:
# Create an index on the 'geom' column
conn.execute(text("""
CREATE INDEX idx_entrance_geom ON train_entrance USING GIST(geom);
"""))

In [26]:
query(conn, "SELECT * FROM train_entrance LIMIT 5;")

,train_station,street_name,street_type,entrance_type,exit_number,geom
0,Aberdeen,Macqueen,St,Ramp,0,0101000020E6100000B61D09EA8ADC6240658CAC885C15...
1,Aberdeen,Macqueen,St,Stairs,0,0101000020E6100000156B5D0F8BDC62409B2113F85C15...
2,Adamstown,Park,Ave,Path,0,0101000020E610000005EBEEF00DF76240C90B44AA8377...
3,Adamstown,Park,Ave,Path,0,0101000020E610000071EC342D0CF76240C75413A78777...
4,Adamstown,St James,Rd,Stairs,0,0101000020E6100000BE2720370DF7624039EACF1B7A77...


### 1.3 Load + clean Train Station Entries Exits data

#### 1.3.1 Load data

In [27]:
# Load the file
entries_exit = pd.read_csv("TrainStationEntriesExits/train-station-entries-exits-data-may-2025.csv")
entries_exit.columns = [col.lower() for col in entries_exit.columns]
entries_exit.columns = entries_exit.columns.str.strip()
entries_exit = entries_exit.rename(columns={'trip': 'trip_count'})
print(entries_exit.columns)

Index(['monthyear', 'station', 'station_type', 'entry_exit', 'trip_count'], dtype='object')


In [28]:
# Check data types
entries_exit.dtypes

monthyear       object
station         object
station_type    object
entry_exit      object
trip_count      object
dtype: object

In [29]:
# Convert MonthYear to date
entries_exit['year_month'] = pd.to_datetime(entries_exit['monthyear'], format='%Y-%m', errors='coerce')
entries_exit = entries_exit.drop(columns = ['monthyear'])        # removing the old monthyear
entries_exit.dtypes

station                 object
station_type            object
entry_exit              object
trip_count              object
year_month      datetime64[ns]
dtype: object

In [30]:
# Check for missing values
entries_exit['trip_count']  = entries_exit['trip_count'].replace('Less than 50', pd.NA)
null_val = entries_exit.isnull().sum()
print(f"Missing values per column:\n{null_val}")

Missing values per column:
station            0
station_type       0
entry_exit         0
trip_count       335
year_month      6282
dtype: int64


#### 1.3.2 Create table for query

In [31]:
conn.execute(text("""
DROP TABLE IF EXISTS train_entry_exit;
CREATE TABLE train_entry_exit (
    year_month    DATE,
    station       VARCHAR(255),
    station_type  VARCHAR(255),
    entry_exit    VARCHAR(50),
    trip_count    INTEGER
);
"""))

In [32]:
entries_exit.to_sql("train_entry_exit", conn, if_exists='append', index=False)

282

In [33]:
query(conn,"SELECT * FROM train_entry_exit LIMIT 5;")

,year_month,station,station_type,entry_exit,trip_count
0,None,Aberdeen Station,train,Entry,NaN
1,None,Aberdeen Station,train,Exit,NaN
2,None,Adamstown Station,train,Entry,2585.0
3,None,Adamstown Station,train,Exit,2391.0
4,None,Albion Park Station,train,Entry,6919.0


### 1.4 Load + clean Opal Patronage data

#### 1.4.1 Load data

In [34]:
import glob

# 1. Find all Opal_Patronage files
file_pattern = "OpalPatronage/Opal_Patronage_202*.txt"
file_list = glob.glob(file_pattern)
print(f"Found {len(file_list)} files:")
# for f in file_list:
#     print(" ", f)

# 2. Read, clean & trim empty columns in each file
dfs = []
for fname in file_list:
    df = pd.read_csv(
        fname,
        sep="|",
        # na_values=["<50"],        # convert "<50" to NaN
        parse_dates=["trip_origin_date"]
    )
    # Drop any column that’s entirely NaN in this file
    df = df.dropna(axis=1, how="all")
    dfs.append(df)

# 3. Concatenate into one DataFrame
if not dfs:
    raise FileNotFoundError(f"No files matched pattern: {file_pattern}")
full_df = pd.concat(dfs, ignore_index=True)

# 4. Inspect the result
print("\nCombined DataFrame shape:", full_df.shape)
display(full_df.head())


Found 2023 files:

Combined DataFrame shape: (1372294, 6)


,trip_origin_date,mode_name,ti_region,tap_hour,Tap_Ons,Tap_Offs
0,2020-01-01,Bus,Chatswood,0,<50,<50
1,2020-01-01,Bus,Macquarie Park,0,<50,<50
2,2020-01-01,Bus,Newcastle and surrounds,0,<50,<50
3,2020-01-01,Bus,North Sydney,0,700,100
4,2020-01-01,Bus,Other,0,4500,3200


#### 1.4.2 Clean data 

In [35]:
# Impute '<50' as NaN, convert to numeric, then median‐impute
# Mark '<50' as missing
full_df['Tap_Ons']  = full_df['Tap_Ons'].replace('<50', pd.NA)
full_df['Tap_Offs'] = full_df['Tap_Offs'].replace('<50', pd.NA)

# Convert to numeric
full_df['Tap_Ons']  = pd.to_numeric(full_df['Tap_Ons'],  errors='coerce')
full_df['Tap_Offs'] = pd.to_numeric(full_df['Tap_Offs'], errors='coerce')

# Compute medians
med_on  = int(full_df['Tap_Ons'].median())
med_off = int(full_df['Tap_Offs'].median())

# Fill missing with medians
full_df['Tap_Ons']  = full_df['Tap_Ons'].fillna(med_on).astype(int)
full_df['Tap_Offs'] = full_df['Tap_Offs'].fillna(med_off).astype(int)

# Confirm no more missing
print("Missing Tap_Ons after imputation:", full_df['Tap_Ons'].isna().sum())
print("Missing Tap_Offs after imputation:", full_df['Tap_Offs'].isna().sum())

Missing Tap_Ons after imputation: 0
Missing Tap_Offs after imputation: 0


#### 1.4.3 Create table for query

In [36]:
conn.execute(text("""
DROP TABLE IF EXISTS opal_patronage;
CREATE TABLE opal_patronage (
    trip_origin_date DATE,
    mode_name        VARCHAR(255),
    ti_region        VARCHAR(255),
    tap_hour         SMALLINT,
    tap_ons          INTEGER,
    tap_offs         INTEGER
);
"""))

In [37]:
# Load the cleaned DataFrame into SQL
# Prepare DataFrame
to_sql_df = full_df[['trip_origin_date','mode_name','ti_region','tap_hour','Tap_Ons','Tap_Offs']].copy()
to_sql_df.columns = ['trip_origin_date','mode_name','ti_region','tap_hour','tap_ons','tap_offs']

In [ ]:
# Write to SQL
to_sql_df.to_sql("opal_patronage", conn, if_exists='append', index=False)

In [ ]:
query(conn,"SELECT * FROM opal_patronage LIMIT 5;")